In [1]:
# Faiss index + metadata(embedding+json) 을 faiss.index파일로 저장
# vector db하겠다면 : pickle 파일로 저장한다고 해도 본직적으로는 벡터 인덱스(검색엔진) + 메타데이터 파일 조합일 뿐, 진짜 벡터 db와 구조적으로 다름
# 즉, 나는 피클파일 등으로 벡터 db를 흉내내는 것

In [3]:
import pandas as pd
df = pd.read_csv('/Users/JooAnLee/final_project/db/merged_news_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   news_id    14087 non-null  object
 1   embedding  14087 non-null  object
 2   wdate      14087 non-null  object
 3   title      14087 non-null  object
 4   article    14087 non-null  object
 5   press      14087 non-null  object
 6   url        14087 non-null  object
dtypes: object(7)
memory usage: 770.5+ KB


In [6]:
import os
import numpy as np
import pickle
import ast

# 임베딩 컬럼 문자열을 np.array로 변환
def parse_embedding(emb_str):
    return np.array(ast.literal_eval(emb_str), dtype=np.float32)

embeddings = np.vstack(df['embedding'].map(parse_embedding).values)

# 4. 메타데이터 추출
meta_cols = ['wdate', 'title', 'article', 'press', 'url']
metadatas = df[meta_cols].to_dict(orient='records')

# 5. 임베딩과 메타데이터를 하나의 딕셔너리로 묶어서 피클로 저장
with open("/Users/JooAnLee/final_project/db/news_vector_db.pkl", "wb") as f:
    pickle.dump({"embeddings": embeddings, "metadatas": metadatas}, f)

print("피클 파일 저장 완료")

피클 파일 저장 완료


In [10]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 8.1 MB/s eta 0:00:00ta 0:00:01


In [12]:
!pip install openai
!pip install requests

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 8.4 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [openai]2m3/4 [openai]


In [17]:
!pip install python-dotenv

In [18]:
import sys
sys.path.append('/Users/JooAnLee/final_project/fastapi/models')  # rag_pipeline.py 경로
from rag_pipeline import NewsTossChatbot
from dotenv import load_dotenv
load_dotenv()  # .env 파일을 읽어서 환경변수로 등록
api_key = os.getenv("OPENAI_API_KEY")
import time

chatbot = NewsTossChatbot(
    vectordb_path="/Users/JooAnLee/final_project/db/news_vector_db.pkl",  # 실제 pkl 경로
    embedding_dim=768
)

start = time.time()
result = chatbot.answer("이스라엘이 이란에 핵 공격했는데 오늘 코스피 주가는 어떻게 될까?", top_k=5)
end = time.time()
print("답변:", result)
print(f"답변 생성 소요 시간: {end - start:.2f}초")


ConnectionError: HTTPConnectionPool(host='15.165.211.100', port=9000): Max retries exceeded with url: /models/embedding (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13797fd30>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
import time

start = time.time()
result = chatbot.answer("민주당에서 대통령 당선됐는데 앞으로 상승할 주식 종목은?", top_k=5)
end = time.time()
print(f"답변 생성 소요 시간: {end - start:.2f}초")
print("답변:", result)

NameError: name 'chatbot' is not defined

# GPT 버전 하기 전에 라마로 했던 2차 버전 저장

In [ ]:
# rag_pipeline.py 2차 코드 임시 저장

import os
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from llama_cpp import Llama
from typing import List, Dict, Any

class CSVDummyVectorDB:
    def __init__(self, csv_path: str):
        df = pd.read_csv(csv_path)
        self.texts = df['content'].tolist()
        self.embeddings = np.vstack(df['embedding'].apply(eval).to_numpy()).astype('float32')
        faiss.normalize_L2(self.embeddings)
        self.index = faiss.IndexFlatIP(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def search(self, query_vec: np.ndarray, top_k: int = 10) -> List[str]:
        faiss.normalize_L2(query_vec)
        scores, indices = self.index.search(query_vec, top_k)
        return [self.texts[i] for i in indices[0]]

class NewsTossChatbot:
    def __init__(
        self,
        csv_path: str = "/app/db/news_v2_vector_202506122113.csv",
        embedding_model_name: str = "snunlp/KR-SBERT-V40K-klueNLI-augSTS",
        reranker_model_name: str = "BAAI/bge-reranker-v2-m3",
        llm_model_path: str = "../models/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf",
        top_k: int = 10,
        rerank_n: int = 5
    ):
        # 1. 임베딩 모델 초기화
        self.embedding_model = SentenceTransformer(embedding_model_name)
        
        # 2. 벡터DB 초기화
        self.vector_db = CSVDummyVectorDB(csv_path)
        
        # 3. Cross-Encoder 리랭커 초기화
        self.reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
        self.reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name).to("cpu")
        self.reranker_model.eval()
        
        # 4. LLaMA3 LLM 초기화
        self.llm = Llama(
            model_path=llm_model_path,
            n_ctx=2048,
            n_threads=4,
            n_gpu_layers=0,
            verbose=False
        )
        
        # 5. 파라미터 설정
        self.top_k = top_k
        self.rerank_n = rerank_n
        
        # 6. 프롬프트 템플릿
        self.prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        당신은 주식 투자자를 위한 뉴스 기반 정보 어시스턴트 챗봇 '뉴스토스'입니다.
        당신의 임무는 실시간 뉴스와 과거 유사사건 뉴스 데이터를 바탕으로,
        - 사용자의 투자 판단에 도움이 되는 정보를 제공하고,
        - 과거 유사사건, 해당 시기의 주가 흐름, 관련 리포트의 핵심 내용을 구체적으로 인용하며,
        - 미래 전망 질문에는 과거 사례를 근거로 신중하게 의견을 제시하는 것입니다.

        답변 작성 시 반드시 다음을 지켜주세요:
        1. 답변 내용 중 포함되는 과거 유사사건의 날짜, 사건명, 당시 주가 흐름(상승/하락/횡보 등), 주요 리포트 내용은 구체적으로 인용하세요.
        2. 미래 전망 질문에는 과거 유사사건을 근거로 논리적인 전망을 제시하세요.
        3. 답변 마지막에는 '⭐️투자 판단은 본인의 책임입니다.⭐️'라는 안내문을 추가하세요.
        4. 답변은 반드시 한글로, 명확하고 간결하게 작성하세요.
        5. 제공된 검색 결과(유사도 높은 과거 뉴스, 주가 데이터, 리포트 등)만 근거로 사용하세요. 근거가 없으면 '근거가 없는데 답변해도 될까? 이건 너의 소중한 돈이 걸린 문제야'라고 하세요.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        검색 결과: {context}
        질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

    def retrieve_docs(self, query: str) -> List[str]:
        """FAISS를 이용한 문서 검색"""
        query_emb = self.embedding_model.encode([query]).astype('float32')
        return self.vector_db.search(query_emb, self.top_k)

    def rerank_docs(self, query: str, docs: List[str]) -> List[str]:
        """Cross-Encoder를 이용한 문서 리랭킹"""
        pairs = [(query, doc) for doc in docs]
        inputs = self.reranker_tokenizer(
            [f"{q} [SEP] {d}" for q, d in pairs],
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        with torch.no_grad():
            scores = self.reranker_model(**inputs).logits.squeeze().cpu().numpy()
        return [doc for _, doc in sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)[:self.rerank_n]]

    def generate_answer(self, context: str, question: str) -> str:
        """LLAMA3 직접 추론"""
        full_prompt = self.prompt_template.format(context=context, question=question)
        output = self.llm(full_prompt, max_tokens=512, stop=["<|eot_id|>"])
        return output['choices'][0]['text']

    def answer(self, query: str) -> Dict[str, str]:
        # 문서 검색 → 리랭킹 → LLM 응답 생성
        docs = self.retrieve_docs(query)
        reranked_docs = self.rerank_docs(query, docs)
        context = "\n\n".join(reranked_docs)
        answer = self.generate_answer(context, query)
        return {"answer": answer, "reranked_docs": reranked_docs}


In [ ]:
# router 2차 코드 임시 저장
from fastapi import APIRouter, HTTPException
from schemas import ChatRequest, ChatResponse
from services.chatbot import get_chatbot_answer

router = APIRouter()

# def로 작성하면 FastAPI가 자동으로 스레드풀에서 병렬 실행
@router.post("/chat", response_model=ChatResponse)
def chat_endpoint(request: ChatRequest):
    try:
        result = get_chatbot_answer(request.question)
        return ChatResponse(answer=result["answer"], reranked_docs=result["reranked_docs"])
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
